# Ejercicio

- Vamos a hacer un algoritmo que cada 200 días rebalancee todos los activos para tener el mismo porcertaje.
- Estamos en modo backtesting: calculamos primero todas las allocations, las insertamos y ejecutamos el backtesting.

1. Descarga el maestro de valores.
2. Descarga todos los datos para cada ticker del maestro. Baja solo el close.
3. Con las series close, crea un datafame donde tengas como columnas los tickers y filas las fechas.
4. Recorre este dataframe cada 200 filas y crea una lista de allocations con valor 1/n_activos. 
5. Envía el post de estos allocations.
6. Usa la API para obtener todas las allocations introducidas.
7. Usa la API para ejecutar el backtesting.
8. Elimina todas las allocations.
9. Refactoriza el código en una clase para el algoritmo y otra para un handler del API.
10. Ejecutalo desde la terminal.

In [ ]:
import pandas as pd
import requests, json

In [ ]:
def allocs_to_frame(json_allocations):
    alloc_list = []
    for json_alloc in json_allocations:
        #print(json_alloc)
        allocs = pd.DataFrame(json_alloc['allocations'])
        allocs.set_index('ticker', inplace=True)
        alloc_serie = allocs['alloc']
        alloc_serie.name = json_alloc['date'] 
        alloc_list.append(alloc_serie)
    all_alloc_df = pd.concat(alloc_list, axis=1).T
    return all_alloc_df


class ApiBmeHandler:
    def __init__(self, market):
        self.url_base = 'https://miax-gateway-jog4ew3z3q-ew.a.run.app'
        self.competi = 'mia_8'
        self.user_key = ''
        self.market = market
    
    def get_ticker_master(self):
        url = f'{self.url_base}/data/ticker_master'
        params = {'competi': self.competi,
                  'market': self.market,
                  'key': self.user_key}
        response = requests.get(url, params)
        tk_master = response.json()
        maestro_df = pd.DataFrame(tk_master['master'])
        return maestro_df
    
    def get_close_data_tck(self, tck):
        url = f'{self.url_base}/data/time_series'
        params = {'market': self.market,
                  'key': self.user_key,
                  'ticker': tck}
        response = requests.get(url, params)
        tk_data = response.json()
        series_data = pd.read_json(tk_data, typ='series')
        return series_data
        
    def get_all_close_data(self):
        datos_close = {}
        maestro_df = self.get_ticker_master()
        for idx, datos in maestro_df.iterrows():
            tck = datos.ticker
            print(tck, end=' ')            
            series_data = self.get_close_data_tck(tck)
            datos_close[tck] = series_data
        return pd.DataFrame(datos_close)
    
    def get_algos(self):
        url = f'{self.url_base}/participants/algorithms'
        params = {'competi': self.competi,
                  'key': self.user_key}
        response = requests.get(url, params)
        algos = response.json()
        if algos:
            algos_df = pd.DataFrame(algos)
            return algos_df

    def post_alloc(self, algo_tag, str_date, allocation):
        url = f'{self.url_base}/participants/allocation'
        url_auth = f'{url}?key={self.user_key}'
        params = {
            'competi': self.competi,
            'algo_tag': algo_tag,
            'market': self.market,
            'date': str_date,
            'allocation': allocation
        }
        response = requests.post(url_auth, data=json.dumps(params))
        print(response.json())

    def get_allocs(self, algo_tag):
        url = f'{self.url_base}/participants/algo_allocations'
        params = {
            'key': self.user_key,
            'competi': self.competi,
            'algo_tag': algo_tag,
            'market': self.market,
        }
        response = requests.get(url, params)
        response_json = response.json()
        df = pd.DataFrame()
        if response_json:
            df = allocs_to_frame(response_json)
        return df

    def exec_backtest(self, algo_tag):
        url = f'{self.url_base}/participants/exec_algo'
        url_auth = f'{url}?key={self.user_key}'
        params = {
            'competi': self.competi,
            'algo_tag': algo_tag,
            'market': self.market,
        }
        response = requests.post(url_auth, data=json.dumps(params))
        if response.status_code == 200:
            exec_data = response.json()
            status = exec_data.get('status')
            res_data = exec_data.get('content')
            trades = None
            if res_data:
                result = pd.Series(res_data['result'])
                trades = pd.DataFrame(res_data['trades'])
            return result, trades
        
    def delete_allocs(self, algo_tag):
        url = f'{self.url_base}/participants/delete_allocations'
        url_auth = f'{url}?key={self.user_key}'

        params = {
            'competi': self.competi,
            'algo_tag': algo_tag,
            'market': self.market,
            }
        response = requests.post(url_auth, data=json.dumps(params))
        print(response.text)
        
        

In [ ]:
# Test of the API Handler
apih = ApiBmeHandler(market='IBEX')

In [ ]:
apih.get_algos()

In [ ]:
apih.get_ticker_master()

In [ ]:
from datetime import datetime

class EqWeightedAlgo:
    
    def __init__(self, market, algo_tag, frec):
        self.apih = ApiBmeHandler(market=market)   
        self.frec = frec
        self.algo_tag = algo_tag
        self.data_close = self.apih.get_all_close_data()
    
    def run_day(self):
        allocs = self.apih.get_allocs(algo_tag=self.algo_tag)
        if allocs.empty:
            # first day
            self.send_current_allocs()
            return
        last_allocs_day = pd.Timestamp(allocs.index[-1])
        last_maket_date = self.data_close.index[-1]
        delta = (last_allocs_day - last_maket_date).days
        if delta >= self.frec:
            print(f'Today is a renal day: {last_allocs_day} delta: {delta}')
        else:
            print(f'No alloc today Delta: {delta}')

    def send_current_allocs(self):
        today_prices = self.data_close.iloc[-1]
        today_maket_day = self.data_close.index[-1]
        allocs = today_prices.dropna()
        allocs[:] = 1 / allocs.shape[0]
        allocs_dic = [
            {'ticker': tck, 'alloc': alloc}
            for tck, alloc in allocs.iteritems()
        ]
        self.apih.post_alloc(
            algo_tag=self.algo_tag,
            str_date=today_maket_day.strftime('%Y-%m-%d'),
            allocation=allocs_dic
        )

    def run(self):
        rbal_dates = self.data_close[::self.frec]
        for day, price_day in rbal_dates.iterrows():
            allocs = price_day.dropna()
            allocs[:] = 1 / allocs.shape[0]
            allocs_dic = [
                {'ticker': tck, 'alloc': alloc}
                for tck, alloc in allocs.iteritems()
            ]    
            self.apih.post_alloc(
                algo_tag=self.algo_tag,
                str_date=day.strftime('%Y-%m-%d'),
                allocation=allocs_dic
            )

    def run_btest(self):
        print(self.apih.get_allocs(algo_tag=self.algo_tag))
        results, trades = self.apih.exec_backtest(algo_tag=self.algo_tag)
        self.apih.delete_allocs(algo_tag=self.algo_tag)
        return results, trades


In [ ]:
eq_1 = EqWeightedAlgo(market='IBEX', algo_tag='test_user_1_miax8_algo1', frec=200)

In [ ]:
# For backtesting run with:
eq_1.run()

In [ ]:
results, trades = eq_1.run_btest()

In [ ]:
# For daily operation, each day you can run:
eq_1 = EqWeightedAlgo(market='IBEX', algo_tag='test_user_1_miax8_algo1', frec=200)
eq_1.run_day()